Here are some preliminary definitions for some functions that we will use repeatedly. Recall that $\psi(N)$ denotes the index of the subgroup $\Gamma_0(N)$ inside $\text{SL}_2(\mathbb{Z})$.

In [53]:
def psi(N): 
    return Gamma0(N).index();
def extend_character(omega): 
    def omega_extended(n):
        if n.is_integer():
            return omega(n)
        else:
            return 0
    return omega_extended

Recall that if $k > 2$, $\chi(-1) = (-1)^k$ and $\gcd(n, N) = 1$, the trace of $T_n$ on the space $S(N, k, \chi)$ is given by $A_1 + A_2 + A_3$ where the individual terms are described below.
We first compute the term $A_1$. It is given by
$$ A_1 = n^{k/2 - 1}\frac{k - 1}{12} \psi(N) \chi(n^{1/2}) $$

In [54]:
def A1(n, k, N, chi):
    chi_extended = extend_character(chi)
    return pow(n, k/2 - 1) * (k - 1)/12 * psi(N) * chi_extended(sqrt(n))

We then compute $A_2$. This term is equal to
$$ A_2 = -\frac{1}{2} \sum_{t^2 < 4n} \frac{\rho^{k - 1} - \bar{\rho}^{k - 1}}{\rho - \bar{\rho}} \sum_m h_w \left(\frac{t^2 - 4n}{f^2}\right)\mu(t, m, n).$$

In [17]:
def mu(t, f, n, N, chi):
    N_f = gcd(N, f)
    
    root_tuples = solve_mod([x^2 - t*x + n == 0], N*N_f)
    roots_mod_N = []
    for root_tuple in root_tuples:
        if root_tuple[0] % N not in roots_mod_N:
            roots_mod_N.append(root_tuple[0])
    
    char_sum = 0
    for root in roots_mod_N:
        if gcd(root, N) == 1:
            char_sum += chi(root)
    
    return psi(N)/psi(N/N_f) * char_sum

def wqfbclassno(D):
    oclassno = pari(f'qfbclassno({D})')
    if D == -4:
        return oclassno/2
    elif D == -3:
        return oclassno/3
    else:
        return oclassno

def A2(n, k, N, chi):
    t_range = range(floor(-2*sqrt(n)) + 1, ceil(2*sqrt(n)))
    t_sum = 0
    for t in t_range:
        f_range = []
        for f in range(1, ceil(sqrt(4*n - t^2))):
            if (t^2 - 4*n) % f^2 == 0:
                if (t^2 - 4*n)/f^2 % 4 == 0 or (t^2 - 4*n)/f^2 % 4 == 1:
                    f_range.append(f)
        f_sum = 0
        for f in f_range:
            f_sum += wqfbclassno((t^2 - 4*n)/f^2) * mu(t, f, n, N, chi)
        
        qroots = solve([x^2 - t*x + n == 0], x, solution_dict = true)
        rho0 = qroots[0][x]
        rho1 = qroots[1][x]
        
        t_sum += (expand(rho0^(k - 1)) - expand(rho1^(k - 1)))/(rho0 - rho1) * f_sum
    return (-1/2) * t_sum

Finally, here is a computation of $A_3$ which is given by
$$ -\frac{1}{2}\sum_{d|n} \min(d, n/d)^{k - 1} \sum_{c} \varphi(\gcd(\tau, N/\tau)))\chi(y)$$
and subsequently the trace formula.

In [18]:
def A3(n, k, N, chi):
    d_range = divisors(n)
    d_sum = 0
    for d in d_range:
        c_range = []
        N_chi = chi.conductor()
        for c in divisors(N):
            if N/N_chi % gcd(c, N/c) == 0 and (n/d - d) % gcd(c, N/c) == 0:
                c_range.append(c)
                
        c_sum = 0
        for c in c_range:
            y = crt([d, n/d], [c, N/c])
            c_sum += euler_phi(gcd(N/c, c))*chi(y)
            
        d_sum += min(d, n/d)^(k - 1) * c_sum
        
    return -(1/2) * d_sum

def trace_formula(n, k, N, chi):
    return A1(n, k, N, chi) + A2(n, k, N, chi) + A3(n, k, N, chi)

Here is an example calculation comparing the result from Sage with the result obtained by using the trace formula.

In [55]:
n = 5; k = 6; N = 35; chi = trivial_character(N);

S = CuspForms(N, k)
T = S.hecke_operator(n)
print(f'The trace as given by Sage is {T.matrix().trace()}.')

s = trace_formula(n, k, N, chi)
print(f'Using the trace formula gives {s}.')


The trace as given by Sage is -24.
Using the trace formula gives -24.
